#Exoplanets Hunting with Machine Learning

 An exoplanet is any planet beyond our solar system. They are detected by astronomers by observing the intensity of their parent starts.

 Exoplanetsare made up of elements similar to those of the planets in our solar system, but their mixes of those elements may differ.

 Exoplanets come in a wide variety pf sizes, from gas giants larger than jupiter to small, rocky planets about as big around as Earth ot Mars.



**History**

` 1. Early claims to have detected exoplanets were made in the 19th century, notably by William Jacob.`

`2. In 1855, William saw deviations in the orrbits within the nearby 70 ophiuchi binary start system.`

`3. The first time we actually detected an exoplanet was in 1917 NASA's JPL but wasn't confirmed.`

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mahoorshams","key":"2b5ec3a9e9700362b646e3b095c12b8d"}'}

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("keplersmachines/kepler-labelled-time-series-data")

print("Path to dataset files:", path)

100%|██████████| 55.9M/55.9M [00:00<00:00, 202MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/keplersmachines/kepler-labelled-time-series-data/versions/3


In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os

path = "/root/.cache/kagglehub/datasets/keplersmachines/kepler-labelled-time-series-data/versions/3"

# List all files and directories
for root, dirs, files in os.walk(path):
    for file in files:
        print(os.path.join(root, file))


/root/.cache/kagglehub/datasets/keplersmachines/kepler-labelled-time-series-data/versions/3/exoTest.csv
/root/.cache/kagglehub/datasets/keplersmachines/kepler-labelled-time-series-data/versions/3/exoTrain.csv


In [8]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/keplersmachines/kepler-labelled-time-series-data/versions/3"

for file in os.listdir(dataset_path):
    print(file)


exoTest.csv
exoTrain.csv


In [9]:
import pandas as pd

train_path = os.path.join(dataset_path, "exoTrain.csv")
train_df = pd.read_csv(train_path)

print(train_df.head())

   LABEL   FLUX.1   FLUX.2   FLUX.3   FLUX.4   FLUX.5   FLUX.6  FLUX.7  \
0      2    93.85    83.81    20.10   -26.98   -39.56  -124.71 -135.18   
1      2   -38.88   -33.83   -58.54   -40.09   -79.31   -72.81  -86.55   
2      2   532.64   535.92   513.73   496.92   456.45   466.00  464.50   
3      2   326.52   347.39   302.35   298.13   317.74   312.70  322.33   
4      2 -1107.21 -1112.59 -1118.95 -1095.10 -1057.55 -1034.48 -998.34   

    FLUX.8  FLUX.9  ...  FLUX.3188  FLUX.3189  FLUX.3190  FLUX.3191  \
0   -96.27  -79.89  ...     -78.07    -102.15    -102.15      25.13   
1   -85.33  -83.97  ...      -3.28     -32.21     -32.21     -24.89   
2   486.39  436.56  ...     -71.69      13.31      13.31     -29.89   
3   311.31  312.42  ...       5.71      -3.73      -3.73      30.05   
4 -1022.71 -989.57  ...    -594.37    -401.66    -401.66    -357.24   

   FLUX.3192  FLUX.3193  FLUX.3194  FLUX.3195  FLUX.3196  FLUX.3197  
0      48.57      92.54      39.32      61.42       5.08  